In [1]:
import datetime
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Santander Product Recommendation

[Kaggle.com](https://www.kaggle.com/c/santander-product-recommendation/details/evaluation)

Цель: рекомендовать **новые** банковские услуги для клиентов в следующем месяце (те, которых не было в предыдущем месяце).

Метрика: MAP@7.

Обучающая выборка с 2015-01-28 до 2016-05-28, тестовая выборка за 2016-06-28. Public/private разделён случайно.

Срок: до 21 декабря.


## Преобработаем данные

Вот так делать не стоит:

In [2]:
# data = pd.read_csv('../input/train_ver2.csv') # Error with different types
# data = pd.read_csv('../input/train_ver2.csv', low_memory=False) # Memory error

In [3]:
data = pd.read_csv('../input/train_ver2.csv', nrows=1000)
data.head()

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,...,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35,2015-01-12,0.0,6,1.0,...,0,0,0,0,0,0,0,0,0,0
1,2015-01-28,1050611,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
2,2015-01-28,1050612,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
3,2015-01-28,1050613,N,ES,H,22,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0
4,2015-01-28,1050614,N,ES,V,23,2012-08-10,0.0,35,1.0,...,0,0,0,0,0,0,0,0,0,0


Укажем типы для некоторых признаков, признаки с датами и возможные пропущенные значения:

In [4]:
data_types = {
    'ncodpers': np.int32, 
    'indrel_1mes': 'object', 
    'conyuemp': 'object', 
    'ind_ahor_fin_ult1': np.int8, 
    'ind_aval_fin_ult1': np.int8, 
    'ind_cco_fin_ult1': np.int8, 
    'ind_cder_fin_ult1': np.int8, 
    'ind_cno_fin_ult1': np.int8, 
    'ind_ctju_fin_ult1': np.int8, 
    'ind_ctma_fin_ult1': np.int8, 
    'ind_ctop_fin_ult1': np.int8, 
    'ind_ctpp_fin_ult1': np.int8, 
    'ind_deco_fin_ult1': np.int8, 
    'ind_deme_fin_ult1': np.int8, 
    'ind_dela_fin_ult1': np.int8, 
    'ind_ecue_fin_ult1': np.int8, 
    'ind_fond_fin_ult1': np.int8, 
    'ind_hip_fin_ult1': np.int8, 
    'ind_plan_fin_ult1': np.int8, 
    'ind_pres_fin_ult1': np.int8, 
    'ind_reca_fin_ult1': np.int8, 
    'ind_tjcr_fin_ult1': np.int8, 
    'ind_valo_fin_ult1': np.int8, 
    'ind_viv_fin_ult1': np.int8, 
    'ind_recibo_ult1': np.int8
}

dates = ['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t']
nas = ['NA', ' NA', '     NA', 'NaN', '         NA']

Теперь можно загружать данные целиком:

In [5]:
data = pd.read_csv('../input/train_ver2.csv', parse_dates=dates, dtype=data_types, na_values=nas)

Если использовать pandas всё-таки не получается, то можно считывать:
- по строкам с помощью csv
- по столбцам с помощью pandas, на ходу преобразуя признаки.

Для удобства обозначим "целевые" переменные и признаки.

In [6]:
target_cols = [
    'ind_ahor_fin_ult1',
    'ind_aval_fin_ult1',
    'ind_cco_fin_ult1',
    'ind_cder_fin_ult1',
    'ind_cno_fin_ult1',
    'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1',
    'ind_ctop_fin_ult1',
    'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1',
    'ind_deme_fin_ult1',
    'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1',
    'ind_fond_fin_ult1',
    'ind_hip_fin_ult1',
    'ind_plan_fin_ult1',
    'ind_pres_fin_ult1',
    'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1',
    'ind_valo_fin_ult1',
    'ind_viv_fin_ult1',
    'ind_nomina_ult1',
    'ind_nom_pens_ult1',
    'ind_recibo_ult1'
]

feature_cols = list(set(data.columns) - set(target_cols) - set(['ncodpers']) - set(dates))

Пример значений признака:

In [7]:
pd.unique(data['indrel_1mes'])

array(['1.0', '1', nan, '3.0', '3', '2', '2.0', '4.0', 'P', '4'], dtype=object)

In [8]:
for col in feature_cols:
    data[col].fillna(-999, inplace=True)

In [9]:
data.indrel_1mes = data.indrel_1mes.apply(lambda x: str(x)[0])

Пропущенные значения есть даже среди "целевых" переменных:

In [10]:
pd.isnull(data[target_cols]).sum(axis=0)

ind_ahor_fin_ult1        0
ind_aval_fin_ult1        0
ind_cco_fin_ult1         0
ind_cder_fin_ult1        0
ind_cno_fin_ult1         0
ind_ctju_fin_ult1        0
ind_ctma_fin_ult1        0
ind_ctop_fin_ult1        0
ind_ctpp_fin_ult1        0
ind_deco_fin_ult1        0
ind_deme_fin_ult1        0
ind_dela_fin_ult1        0
ind_ecue_fin_ult1        0
ind_fond_fin_ult1        0
ind_hip_fin_ult1         0
ind_plan_fin_ult1        0
ind_pres_fin_ult1        0
ind_reca_fin_ult1        0
ind_tjcr_fin_ult1        0
ind_valo_fin_ult1        0
ind_viv_fin_ult1         0
ind_nomina_ult1      16063
ind_nom_pens_ult1    16063
ind_recibo_ult1          0
dtype: int64

In [11]:
for col in target_cols:
    data[col].fillna(0, inplace=True)

Закодируем числами категориальные переменные:

In [12]:
encode_features = [
    'conyuemp', 
    'indfall', 
    'tipodom', 
    'indext', 
    'indresi', 
    'pais_residencia', 
    'segmento',
    'canal_entrada', 
    'indrel_1mes', 
    'sexo', 
    'ind_empleado', 
    'nomprov', 
    'tiprel_1mes'
]

encoders = {}
for col in encode_features:
    le = LabelEncoder()
    le.fit(data[col])
    data[col] = le.fit_transform(data[col])
    encoders[col] = le

Сохраним датасет:

In [13]:
data.to_csv('../input/train_preprocessed.csv', index=False)

Аналогичные преобразования нужно произвести с тестовым датасетом (не забываем, что Label Encoder нужно использовать уже обученный).

In [16]:
data = pd.read_csv('../input/test_ver2.csv', parse_dates=dates, dtype=data_types, na_values=nas)

In [17]:
for col in feature_cols:
    data[col].fillna(-999, inplace=True)
    
data.indrel_1mes = data.indrel_1mes.apply(lambda x: str(x)[0])
    
for col in encode_features:
    le = encoders[col]
    data[col] = le.transform(data[col])

In [18]:
data.to_csv('../input/test_preprocessed.csv', index=False)

## Обучение, валидация

Метрика для оценки результата:

In [2]:
def apk(actual, predicted, k=10):

    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(actual, predicted, k=10):
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

Теперь можно считывать данные экономичнее:

In [3]:
data_types = {
    'ncodpers': np.int32, 
    'conyuemp': np.int8,
    'indfall': np.int8,
    'tipodom': np.int8,
    'indext': np.int8,
    'indresi': np.int8,
    'pais_residencia': np.int8,
    'segmento': np.int8,
    'canal_entrada': np.int8,
    'indrel_1mes': np.int8,
    'sexo': np.int8,
    'ind_empleado': np.int8,
    'nomprov': np.int8,
    'tiprel_1mes': np.int8,
    'ind_ahor_fin_ult1': np.int8, 
    'ind_aval_fin_ult1': np.int8, 
    'ind_cco_fin_ult1': np.int8, 
    'ind_cder_fin_ult1': np.int8, 
    'ind_cno_fin_ult1': np.int8, 
    'ind_ctju_fin_ult1': np.int8, 
    'ind_ctma_fin_ult1': np.int8, 
    'ind_ctop_fin_ult1': np.int8, 
    'ind_ctpp_fin_ult1': np.int8, 
    'ind_deco_fin_ult1': np.int8, 
    'ind_deme_fin_ult1': np.int8, 
    'ind_dela_fin_ult1': np.int8, 
    'ind_ecue_fin_ult1': np.int8, 
    'ind_fond_fin_ult1': np.int8, 
    'ind_hip_fin_ult1': np.int8, 
    'ind_plan_fin_ult1': np.int8, 
    'ind_pres_fin_ult1': np.int8, 
    'ind_reca_fin_ult1': np.int8, 
    'ind_tjcr_fin_ult1': np.int8, 
    'ind_valo_fin_ult1': np.int8, 
    'ind_nomina_ult1': np.int8,
    'ind_nom_pens_ult1': np.int8,
    'ind_viv_fin_ult1': np.int8, 
    'ind_recibo_ult1': np.int8
}

dates = ['fecha_dato', 'fecha_alta', 'ult_fec_cli_1t']

In [4]:
data = pd.read_csv('../input/train_preprocessed.csv', parse_dates=dates, dtype=data_types)

In [5]:
# data_test = pd.read_csv('../input/test_preprocessed.csv', parse_dates=dates, dtype=data_types)

In [6]:
target_cols = [
    'ind_ahor_fin_ult1',
    'ind_aval_fin_ult1',
    'ind_cco_fin_ult1',
    'ind_cder_fin_ult1',
    'ind_cno_fin_ult1',
    'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1',
    'ind_ctop_fin_ult1',
    'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1',
    'ind_deme_fin_ult1',
    'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1',
    'ind_fond_fin_ult1',
    'ind_hip_fin_ult1',
    'ind_plan_fin_ult1',
    'ind_pres_fin_ult1',
    'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1',
    'ind_valo_fin_ult1',
    'ind_viv_fin_ult1',
    'ind_nomina_ult1',
    'ind_nom_pens_ult1',
    'ind_recibo_ult1'
]


feature_cols = list(set(data.columns) - set(target_cols) - set(['ncodpers']) - set(dates))

Разделим на обучение/валидацию по времени, заодно обозначим предпоследний месяц для фильтрации предсказанных услуг.

In [7]:
# valid
train_index = np.array((data.fecha_dato == datetime.datetime(2016, 4, 28)) | \
                (data.fecha_dato == datetime.datetime(2015, 4, 28)) | \
                (data.fecha_dato == datetime.datetime(2015, 5, 28)))
valid_index = np.array(data.fecha_dato == datetime.datetime(2016, 5, 28))

last_month_index = np.array(data.fecha_dato == datetime.datetime(2016, 4, 28))

In [8]:
# # test
# train_index = np.array((data.fecha_dato == datetime.datetime(2016, 4, 28)) | \
#                 (data.fecha_dato == datetime.datetime(2015, 4, 28)) | \
#                 (data.fecha_dato == datetime.datetime(2015, 6, 28)) | \
#                 (data.fecha_dato == datetime.datetime(2015, 5, 28)) | \
#                 (data.fecha_dato == datetime.datetime(2016, 5, 28)))

# last_month_index = np.array(data.fecha_dato == datetime.datetime(2016, 5, 28))

Другой вариант кросс-валидации:
- Обучаемся на первых 4 месяцах, оцениваем результат по 5 месяцу.
- Обучаемся на первых 5 месяцах, оцениваем результат по 6 месяцу.
- Обучаемся на первых 6 месяцах, оцениваем результат по 7 месяцу.
- ...
- Усредняем полученные оценки качества.

Выделим заранее необходимые для обучения/предсказания/оценки данные, чтобы отказаться от data.

In [9]:
# valid
ncodpers = np.array(data.ix[valid_index, 'ncodpers'])
last_month_target = np.array(data.ix[last_month_index, ['ncodpers'] + list(target_cols)])

In [10]:
# # test
# ncodpers = np.array(data_test['ncodpers'])
# last_month_target = np.array(data.ix[last_month_index, ['ncodpers'] + list(target_cols)])

Откажемся от pandas:

In [14]:
# valid
X_train = np.array(data.ix[train_index, feature_cols])
X_valid = np.array(data.ix[valid_index, feature_cols])
y_train = np.array(data.ix[train_index, target_cols])
y_valid = np.array(data.ix[valid_index, target_cols])

In [15]:
# # test
# X_train = np.array(data.ix[train_index, feature_cols])
# y_train = np.array(data.ix[train_index, target_cols])

# X_test = np.array(data_test[feature_cols])

Сохраним данные после разделения.

In [16]:
# valid
np.save('../input/X_train.npy', X_train)
np.save('../input/X_valid.npy', X_valid)
np.save('../input/y_train.npy', y_train)
np.save('../input/y_valid.npy', y_valid)

np.save('../input/ncodpers.npy', ncodpers)
np.save('../input/last_month_target.npy', last_month_target)

In [12]:
# # test
# np.save('../input/X_train.npy', X_train)
# np.save('../input/y_train.npy', y_train)

# np.save('../input/X_test.npy', X_test)

# np.save('../input/ncodpers.npy', ncodpers)
# np.save('../input/last_month_target.npy', last_month_target)

Теперь можно начинать с нуля.

In [3]:
# valid
X_train = np.load('../input/X_train.npy')
X_valid = np.load('../input/X_valid.npy')
y_train = np.load('../input/y_train.npy')
y_valid = np.load('../input/y_valid.npy')

ncodpers = np.load('../input/ncodpers.npy')
last_month_target = np.load('../input/last_month_target.npy')

Обучим модель и оценим результат:

In [17]:
clf = RandomForestClassifier(n_estimators=50, max_depth=20, criterion='entropy', n_jobs=-1, random_state=5)
clf.fit(X_train, y_train)
y_pred = np.array(clf.predict_proba(X_valid))[:, :, 1].T

Для каждого клиента запомним услуги, использованные в предпоследнем месяце (их не нужно рекомендовать).

In [18]:
last_products = {}
for i in range(last_month_target.shape[0]):
    row = last_month_target[i]
    cust_id = row[0]
    used_products = set(np.where(row[1:] == 1)[0])
    last_products[cust_id] = used_products

Построим рекомендации для последнего месяца и истинные ответы.

In [19]:
y_pred = np.argsort(y_pred, axis=1)
y_pred = np.fliplr(y_pred)

preds = []
trues = []
for i in range(y_pred.shape[0]):
    cust_id = ncodpers[i]
    used_products = last_products.get(cust_id, {})
    
    pred_top_products = []
    for product_id in y_pred[i]:
        if product_id not in used_products:
            pred_top_products.append(product_id)
        if len(pred_top_products) == 7:
            break
    
    products = np.arange(0, len(y_valid[i]))[y_valid[i].astype(bool)]
    true_top_products = []
    for product_id in products:
        if product_id not in used_products:
            true_top_products.append(product_id)
    
    preds.append(pred_top_products)
    trues.append(true_top_products)

Оценим качество:

In [20]:
mapk(trues, preds, k=7)

0.020181858343899257

Это много или мало? На самом деле совсем мало новых услуг можно предложить людям в новом месяце. [Если](https://www.kaggle.com/sudalairajkumar/santander-product-recommendation/maximum-possible-score/discussion) сделать идеальное предсказание для мая 2016, то можно получить 0.0319. Поэтому результат так далёк от 1.0.

Теперь можно повторить то же самое с разделением на обучающую/тестовую выборку и сделать сабмит.

## Подготовка сабмита

In [2]:
# test
X_train = np.load('../input/X_train.npy')
y_train = np.load('../input/y_train.npy')

X_test = np.load('../input/X_test.npy')

ncodpers = np.load('../input/ncodpers.npy')
last_month_target = np.load('../input/last_month_target.npy')

In [3]:
clf = RandomForestClassifier(n_estimators=50, max_depth=20, criterion='entropy', n_jobs=-1, random_state=5)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=-1, oob_score=False, random_state=5,
            verbose=0, warm_start=False)

In [8]:
y_pred = np.array(clf.predict_proba(X_test))[:, :, 1].T

Теперь уже запомним услуги последнего месяца обучающей выборки:

In [5]:
last_products = {}
for i in range(last_month_target.shape[0]):
    row = last_month_target[i]
    cust_id = row[0]
    used_products = set(np.where(row[1:] == 1)[0])
    last_products[cust_id] = used_products

Снова используем названия услуг:

In [6]:
target_cols = np.array([
    'ind_ahor_fin_ult1',
    'ind_aval_fin_ult1',
    'ind_cco_fin_ult1',
    'ind_cder_fin_ult1',
    'ind_cno_fin_ult1',
    'ind_ctju_fin_ult1',
    'ind_ctma_fin_ult1',
    'ind_ctop_fin_ult1',
    'ind_ctpp_fin_ult1',
    'ind_deco_fin_ult1',
    'ind_deme_fin_ult1',
    'ind_dela_fin_ult1',
    'ind_ecue_fin_ult1',
    'ind_fond_fin_ult1',
    'ind_hip_fin_ult1',
    'ind_plan_fin_ult1',
    'ind_pres_fin_ult1',
    'ind_reca_fin_ult1',
    'ind_tjcr_fin_ult1',
    'ind_valo_fin_ult1',
    'ind_viv_fin_ult1',
    'ind_nomina_ult1',
    'ind_nom_pens_ult1',
    'ind_recibo_ult1'
])

Подготовим рекомендации (теперь надо не массив индексов, а конкретные названия услуг).

In [9]:
y_pred = np.argsort(y_pred, axis=1)
y_pred = np.fliplr(y_pred)

preds = []
for i in range(y_pred.shape[0]):
    cust_id = ncodpers[i]
    used_products = last_products.get(cust_id, {})
    
    pred_top_products = []
    for product_id in y_pred[i]:
        if product_id not in used_products:
            pred_top_products.append(product_id)
        if len(pred_top_products) == 7:
            break 
   
    preds.append(np.array(pred_top_products))
    
final_preds = [' '.join(list(target_cols[pred])) for pred in preds]
out = pd.DataFrame({'ncodpers': ncodpers, 'added_products': final_preds})
out.to_csv('../output/submission.csv', index=False)

## Ссылки:

Можно посмотреть:

- [Визуализация 1](https://www.kaggle.com/apryor6/santander-product-recommendation/detailed-cleaning-visualization-python/comments)
- [Визуализация 2](https://www.kaggle.com/yifanxie/santander-product-recommendation/santander-products-visualisation)
- [Уменьшение датасета](https://www.kaggle.com/jturkewitz/santander-product-recommendation/reduce-size-of-dataset-to-1-gb/comments)
- [Скрипт 1](https://www.kaggle.com/tanlikesmath/santander-product-recommendation/when-less-is-more-1/code)
- [Скрипт 2](https://www.kaggle.com/sudalairajkumar/santander-product-recommendation/rf-multilabel-framework-lb-0-022475/code)

- [Простой гайд по AWS](https://github.com/emilkayumov/aws-jupyter)
- [Expedia Hotel Recommendations](https://www.kaggle.com/c/expedia-hotel-recommendations) (стоит и другие конкурсы поискать).

и прочее на форуме и в скриптах.

Не будем забывать о прошлом конкурсе [Santander](https://www.kaggle.com/c/santander-customer-satisfaction/leaderboard/private) (изменение позиций в лидерборде).